In [1]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
url="https://decantworldbd.com/product-category/mens-perfume/"
head2=({"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36",'Accept-Language': 'en-US, en;q=0.5'})


In [3]:
request=requests.get(url,headers=head2)

In [4]:
html_content=request.content

In [5]:
soup=BeautifulSoup(html_content,"html.parser")

In [6]:
links=soup.find_all("h3")

In [7]:
link=links[0].find("a").get("href")

In [8]:
link

'https://decantworldbd.com/product/acqua-di-gio-edt-for-men-by-giorgio-armani/'

In [9]:
head2=({"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36",'Accept-Language': 'en-US, en;q=0.5'})

In [10]:
new_request=requests.get(link,headers=head2)
new_content=new_request.content

In [11]:
new_soup=BeautifulSoup(new_content,"html.parser")

In [12]:
name=new_soup.find("h1").text.strip()

In [13]:
price=new_soup.find("p",class_="price").text

In [14]:
def get_name(soup):
    
    
    try:
        name=new_soup.find("h1").text.strip()
        
    except AttributeError:
        
        name=""
        
    return name

def get_price(soup):
    
    try:
        
        price=new_soup.find("p",class_="price").text
        
    except AttributeError:
        
        price=""
        
    return price

In [15]:
if __name__ == '__main__':

    # add your user agent 
    url="https://decantworldbd.com/product-category/mens-perfume/"
    headers=({"User-Agent": "Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:47.0) Gecko/20100101 Firefox/47.0",'Accept-Language': 'en-US, en;q=0.5'})

   
    # HTTP Request
    webpage = requests.get(url, headers=headers)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links=soup.find_all("h3")

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.find("a").get("href"))

    d = {"title":[], "price":[]}
    
    # Loop for extracting product details from each link 
    for link in links_list:
        new_webpage = requests.get(link,headers=head2)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_name(new_soup))
        d['price'].append(get_price(new_soup))
        
    
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)

In [16]:
amazon_df

,title,price
0,Acqua DI Gio EDT for Men By Giorgio Armani,"৳ 500.00 – ৳ 2,700.00"
1,Acqua DI Gio Profondo EDP for Men by Giorgio A...,"৳ 650.00 – ৳ 3,750.00"
2,Acqua DI Gio Profumo EDP for Men By Giorgio Ar...,"৳ 650.00 – ৳ 3,750.00"
3,Afnan 9am Dive EDP for Men and Women,"৳ 300.00 – ৳ 1,650.00"
4,Afnan 9am EDP for Women,"৳ 280.00 – ৳ 1,500.00"
5,Afnan 9pm for Men EDP,"৳ 280.00 – ৳ 1,500.00"
6,Afnan Supremacy in Oud EDP for women and men,"৳ 400.00 – ৳ 2,250.00"
7,Afnan Supremacy Not Only Intense EDP for Men,"৳ 450.00 – ৳ 2,610.00"
8,Afnan Turathi Blue EDP for Men,"৳ 350.00 – ৳ 1,950.00"
9,Afnan Turathi Brown EDP for Men,"৳ 370.00 – ৳ 2,100.00"


In [19]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

is_scrapping = True
current_page = 1
scrapped_data = []

# Define functions for extracting name and price
def get_name(soup):
    return soup.find("h1", class_="product_title").text.strip()

def get_price(soup):
    price_element = soup.find("p", class_="price").text.strip()
    return price_element

# Add your user agent
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:47.0) Gecko/20100101 Firefox/47.0",
    'Accept-Language': 'en-US, en;q=0.5'
}

# Initialize a dictionary to store product data
d = {"title": [], "price": []}

while is_scrapping:
    url = f"https://decantworldbd.com/product-category/mens-perfume/page/{current_page}/"

    # HTTP Request
    webpage = requests.get(url, headers=headers)

    # Parse the page content with BeautifulSoup
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Find product links
    links = soup.find_all("h3")

    # Store the links in a list
    links_list = [link.find("a").get("href") for link in links]

    # Extract product details from each link
    for link in links_list:
        new_webpage = requests.get(link, headers=headers)
        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Get and append product details
        d['title'].append(get_name(new_soup))
        d['price'].append(get_price(new_soup))

    # Check if there's a next page link
    next_page = soup.find("a", class_="next page-numbers")
    if next_page:
        current_page += 1  # Move to the next page
    else:
        is_scrapping = False  # Stop if no next page

# Create a DataFrame from the scraped data
perfume_df = pd.DataFrame.from_dict(d)

# Clean up DataFrame and save to CSV
perfume_df = perfume_df.dropna(subset=['title'])
perfume_df = perfume_df.dropna(subset=['title'])
perfume_df.to_csv("perfume_df.csv", header=True, index=False)

print("Scraping complete. Data saved to perfume_df.csv")


Scraping complete. Data saved to perfume_df.csv


In [20]:
perfume_df


,title,price
0,Acqua DI Gio EDT for Men By Giorgio Armani,"৳ 500.00 – ৳ 2,700.00"
1,Acqua DI Gio Profondo EDP for Men by Giorgio A...,"৳ 650.00 – ৳ 3,750.00"
2,Acqua DI Gio Profumo EDP for Men By Giorgio Ar...,"৳ 650.00 – ৳ 3,750.00"
3,Afnan 9am Dive EDP for Men and Women,"৳ 300.00 – ৳ 1,650.00"
4,Afnan 9am EDP for Women,"৳ 280.00 – ৳ 1,500.00"
...,...,...
204,YSL Y EDP for Men,"৳ 650.00 – ৳ 3,750.00"
205,Zara Vibrant Leather Eclat de Bergamote for Men,"৳ 350.00 – ৳ 1,550.00"
206,Zara Vibrant Leather Summer EDP for men,৳ 170.00 – ৳ 900.00
207,Zeleny by Ahmed Al Maghribi for Men and Women,"৳ 250.00 – ৳ 1,350.00"
